# High Resolution Conflict Forecasting with Spatial Convolutions and Long Short-Term Memory

## Replication Archive

[Benjamin J. Radford](https://www.benradford.com)    
Assistant Professor  
UNC Charlotte  
bradfor7@uncc.edu  

This file replicates the following tables and figures:

* Table 3

## Imports and seeds

In [1]:
import sys
import os
import gc
import logging

import pandas as pd
import numpy as np
from datetime import datetime

from sklearn.ensemble import RandomForestRegressor
from joblib import dump, load

from itertools import product
from math import isnan

import views
from views import Period, Model, Downsampling
from views.utils.data import assign_into_df
from views.apps.transforms import lib as translib
from views.apps.evaluation import lib as evallib, feature_importance as fi
from views.apps.model import api
from views.apps.extras import extras

import keras
from keras.models import Model
from keras.layers import Input, ConvLSTM2D, Activation, Conv3D, BatchNormalization, Dropout, Bidirectional, GaussianNoise, Multiply
from keras import optimizers

import tensorflow as tf

import random
import geoplot as gplt
import contextily as ctx

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import AxesGrid
from mpl_toolkits.axes_grid1 import make_axes_locatable

from numpy.random import seed
seed(1234)
tf.random.set_seed(1234)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
cols_ids = [
    "col_idx",
    "row_idx",
    "pg_id",
    "year",
    "month",
    "year_idx",
    "month_idx",
    "year_month_idx"]

cols_feats = [
    "ln_ged_best_sb",
    "pgd_bdist3",
    "pgd_capdist",
    "pgd_agri_ih",
    "pgd_pop_gpw_sum",
    "pgd_ttime_mean",
    "spdist_pgd_diamsec",
    "pgd_pasture_ih",
    "pgd_savanna_ih",
    "pgd_forest_ih",
    "pgd_urban_ih",
    "pgd_barren_ih",
    "pgd_gcp_mer"
]

cols_lags = [
    "delta_1",
    "delta_2",
    "delta_3",
    "delta_4",
    "delta_5",
    "delta_6",
    "delta_7"
]


## Load all attention values

In [3]:
attn_preds = np.load("../data/attention_model/attention_values.npy")

## Compute means and produce Table 3

In [4]:
feat_imp = list(np.mean(attn_preds, axis=(0,1,2,3)))

for ff,ww in sorted(list(zip(cols_feats+["missing_indicator"],feat_imp)), key = lambda x:x[1], reverse=True ):
    print(f"\\verb@ {ff} @ & {ww:.3f} \\\\")

\verb@ ln_ged_best_sb @ & 0.284 \\
\verb@ pgd_pop_gpw_sum @ & 0.271 \\
\verb@ pgd_urban_ih @ & 0.207 \\
\verb@ pgd_ttime_mean @ & 0.051 \\
\verb@ pgd_agri_ih @ & 0.040 \\
\verb@ pgd_gcp_mer @ & 0.035 \\
\verb@ pgd_forest_ih @ & 0.029 \\
\verb@ spdist_pgd_diamsec @ & 0.017 \\
\verb@ pgd_barren_ih @ & 0.016 \\
\verb@ pgd_bdist3 @ & 0.014 \\
\verb@ pgd_savanna_ih @ & 0.012 \\
\verb@ pgd_pasture_ih @ & 0.011 \\
\verb@ missing_indicator @ & 0.010 \\
\verb@ pgd_capdist @ & 0.010 \\
